In [1]:
import numpy as np
import pandas as pd

In [2]:

temp_df = pd.read_csv('IMDB Dataset.csv')

In [3]:
df = temp_df.iloc[:10000]

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.drop_duplicates(inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_17028\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [6]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [7]:
df['review'] = df['review'].apply(lambda x:x.lower())

C:\Users\HP\AppData\Local\Temp\ipykernel_17028\740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [10]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


C:\Users\HP\AppData\Local\Temp\ipykernel_17028\3841627977.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [11]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. <br /><br />the f...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [12]:
import gensim

**You've imported the gensim library, which is a popular library for natural language processing and text analysis tasks, particularly for working with word embeddings and topic modeling. gensim provides tools for training and using word embeddings like Word2Vec and Doc2Vec, as well as functions for topic modeling with methods like Latent Dirichlet Allocation (LDA).**

In [13]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

1. sent_tokenize from NLTK:
   * sent_tokenize is used for sentence tokenization, which means breaking a text into individual sentences.
    * It takes a block of text as input and returns a list of sentences.
    
2. simple_preprocess from gensim:
   * simple_preprocess is a utility function for text preprocessing that performs basic preprocessing steps such as tokenization, lowercasing, and removing punctuation.
    * It takes a string as input and returns a list of preprocessed tokens (words).

In [14]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))
    

In [15]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [16]:
model.build_vocab(story)

In [17]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5935553, 6424740)

In [18]:
len(model.wv.index_to_key)

31846

In [19]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [20]:
document_vector(df['review'].values[0])

array([-0.2410262 ,  0.34991115,  0.07217623,  0.03344386,  0.01770463,
       -0.59102666,  0.20887274,  0.9412403 , -0.05803785, -0.17358612,
       -0.07372425, -0.55649364,  0.00865161,  0.3903537 ,  0.05299453,
       -0.29992655,  0.13905257, -0.33307177, -0.0268338 , -0.6186111 ,
       -0.06469975,  0.09567872,  0.3057215 , -0.06521441,  0.054617  ,
       -0.02705205, -0.27546784, -0.25594318, -0.34188765,  0.03586407,
        0.31191364, -0.01321147,  0.21706605, -0.41404295, -0.08733148,
        0.42363793, -0.00166671, -0.3188812 , -0.30863345, -0.78469664,
       -0.24822423, -0.07277132, -0.11642195,  0.09557318,  0.5143803 ,
       -0.208121  , -0.27382684, -0.06578827,  0.11419652,  0.10689662,
        0.09756421, -0.37686628, -0.31044698, -0.06043475, -0.09358449,
        0.09896173,  0.31208342,  0.04872354, -0.38573456,  0.11380082,
       -0.02272581,  0.0786477 ,  0.00750763,  0.07119747, -0.47490755,
        0.21638776,  0.05755654,  0.26009113, -0.5586753 ,  0.56

In [21]:
from tqdm import tqdm

**You've imported the tqdm library, which is a popular library for adding progress bars to loops and iterations in Python. The name "tqdm" stands for "taqaddum" in Arabic, which means "progress."**

In [22]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))


100%|██████████| 9983/9983 [13:34<00:00, 12.26it/s]


In [25]:
X = np.array(X)

In [26]:
X[0]

array([-0.2410262 ,  0.34991115,  0.07217623,  0.03344386,  0.01770463,
       -0.59102666,  0.20887274,  0.9412403 , -0.05803785, -0.17358612,
       -0.07372425, -0.55649364,  0.00865161,  0.3903537 ,  0.05299453,
       -0.29992655,  0.13905257, -0.33307177, -0.0268338 , -0.6186111 ,
       -0.06469975,  0.09567872,  0.3057215 , -0.06521441,  0.054617  ,
       -0.02705205, -0.27546784, -0.25594318, -0.34188765,  0.03586407,
        0.31191364, -0.01321147,  0.21706605, -0.41404295, -0.08733148,
        0.42363793, -0.00166671, -0.3188812 , -0.30863345, -0.78469664,
       -0.24822423, -0.07277132, -0.11642195,  0.09557318,  0.5143803 ,
       -0.208121  , -0.27382684, -0.06578827,  0.11419652,  0.10689662,
        0.09756421, -0.37686628, -0.31044698, -0.06043475, -0.09358449,
        0.09896173,  0.31208342,  0.04872354, -0.38573456,  0.11380082,
       -0.02272581,  0.0786477 ,  0.00750763,  0.07119747, -0.47490755,
        0.21638776,  0.05755654,  0.26009113, -0.5586753 ,  0.56

In [27]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

**You've imported the LabelEncoder class from scikit-learn's preprocessing module. LabelEncoder is a useful tool for preprocessing categorical labels in your dataset by converting them into numerical values. It assigns a unique integer to each category in your data, making it suitable for various machine learning algorithms that require numeric input for labels.**

In [28]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

 **You've imported the train_test_split function from scikit-learn's model_selection module. train_test_split is a fundamental function used in machine learning to split your dataset into training and testing sets. This division allows you to train your model on one subset of the data and evaluate its performance on another, unseen subset.**

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

**1.  RandomForestClassifier: This is a class from the sklearn's ensemble module, and it represents the Random Forest classifier, which is an ensemble learning method for classification tasks. Random Forest is known for its robustness and ability to handle both categorical and numerical features.**

**2.  accuracy_score: This is a function from the metrics module in sklearn. It's used to calculate the accuracy of your classification model, which measures the fraction of correctly predicted labels.After making predictions with your classifier, you can use accuracy_score to evaluate its performance:**

In [31]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7746619929894842